# Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [6]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), download_if_missing=True, data_home='data')
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), download_if_missing=True, data_home='data')

## Vectorización

In [75]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [8]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [77]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [10]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [70]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [20]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [21]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [23]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [24]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [25]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [26]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [27]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750], dtype=int64)

In [28]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [29]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [30]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


## Modelo de clasificación Naïve Bayes

In [31]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [32]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [33]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


In [38]:
import random

## Parte 1
Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.


In [202]:
# Se crea una clase que guarda los valores 
    # ID de un documento
    # su vector de similitud de coseno
    # dos vectores ordenados de la similitud con un documento y su respectivo ID.
# La clase tambien tiene una funcion para encontrar la N cantidad de documentos similares (y guardar sus respectivos datos)
class Document():
    def __init__(self, idx):
        self.idx = idx
        self.cossim = cosine_similarity(X_train[self.idx], X_train)
        self.sim = np.sort(self.cossim)[::-1]
        self.docsim = np.argsort(self.cossim)[::-1]
        self.class_type = newsgroups_train.target_names[y_train[self.idx]]
        
    def similar_documents(self, amount):
        print(f"Analizando documento {self.idx}, de categorias [{self.class_type}]")
        print("Los documentos mas similares son: ")
        
        similar_docs = []
                    
        for i in range(1, amount + 1):
            idx = self.docsim[:][0][-1-i]
            sim = self.sim[:][0][-1-i]
            class_type = newsgroups_train.target_names[y_train[idx]]
            print (f"{100 * sim:.2f}% similaridad con el documento {idx} de clases [{class_type}]")
            
            similar_docs.append({
                'idx': idx,
                'sim': sim,
                'class_type': class_type
            })
            
        return similar_docs

In [227]:
# Creamos una lista para guardar 5 documentos aleatorios
documents = []
    
# Creamos una de las clases anteriores para cada uno de esos 5 documentos aleatorios
for i in range(5):
    documents.append(
        Document(random.randint(0, len(newsgroups_train.data) - 1))
    )
    
# Guardamos las similitudes en otra lista
similarities = []
    
for i in range(len(documents)):
    similarities.append(documents[i].similar_documents(5))
    print()

Analizando documento 2416, de categorias [alt.atheism]
Los documentos mas similares son: 
42.01% similaridad con el documento 2195 de clases [alt.atheism]
36.69% similaridad con el documento 4137 de clases [talk.religion.misc]
36.51% similaridad con el documento 2147 de clases [alt.atheism]
32.82% similaridad con el documento 6352 de clases [talk.religion.misc]
29.05% similaridad con el documento 5535 de clases [talk.religion.misc]

Analizando documento 10271, de categorias [talk.politics.mideast]
Los documentos mas similares son: 
65.36% similaridad con el documento 4507 de clases [talk.politics.mideast]
45.25% similaridad con el documento 10765 de clases [talk.politics.mideast]
44.96% similaridad con el documento 7753 de clases [talk.politics.mideast]
44.03% similaridad con el documento 503 de clases [talk.politics.mideast]
43.27% similaridad con el documento 10328 de clases [talk.politics.mideast]

Analizando documento 2361, de categorias [misc.forsale]
Los documentos mas similares 

Basandonos unicamente en las clases de los documentos, se puede ver con claridad que todos los articulos comparten clases en comun, y algunos difieren a lo mucho en 1 o 2 clases.

Para poder verificar el contenido de los mismos, tendriamos que leer los 5 documentos originales y los 5 documentos mas similares a cada uno (es decir, 30 documentos), y alguno de ellos son muy extensos. Vamos a emplear ChatGPT para hacer esta tarea mas sencilla.

Nuestro primer prompt es el siguiente:

> En el siguiente mensaje que te voy a escribir, podras leer un texto. Ese es el texto original. Como respuesta al texto original, solo responde "Entendido". En siguientes mensajes, te voy a mandar otros textos. Como respuesta a cada texto adicional, quiero que me digas qué tanto en común tiene el documento enviado y el documento original (una breve descripción de no más de 1 párrafo), y adicionalmente des un porcentaje de similitud aproximado (sin necesidad de explicar nada)

Y los posteriores prompts son los respectivos textos de los documentos.

### Documento 1


Respuesta al documento 1:

> El documento similar discute la valoración de la libertad, sugiriendo que aunque muchas personas consideran que la libertad es valiosa, no es universalmente aceptada como un valor objetivo. Ambos textos tratan sobre la libertad y su valor percibido, pero difieren en sus conclusiones sobre su objetividad.

> Porcentaje de similitud: 70%

Respuesta al documento 2:

> El documento similar aborda la relación entre la ciencia, los valores y la moralidad objetiva, sugiriendo que la ciencia, aunque basada en hechos observables, está influenciada por los valores de los observadores. Además, discute la idea de una moralidad objetiva en el contexto de la existencia de un Dios omnipotente. Ambos textos tratan sobre la objetividad en diferentes contextos (valores y moralidad), pero con enfoques distintos.

> Porcentaje de similitud: 60%

Respuesta al documento 3:

> El documento similar expresa una opinión subjetiva sobre la libertad y discute la falta de consenso sobre su significado, cuestionando cómo un valor puede ser objetivo si su definición no lo es. Ambos textos tratan sobre la libertad y su valoración, pero el documento similar enfatiza más la subjetividad y la variabilidad en la interpretación del término.

> Porcentaje de similitud: 75%

Respuesta al documento 4:

> El documento similar critica el uso de la palabra "objetivo" en relación con "valores" y señala la subjetividad inherente a conceptos como el valor y la moralidad. Sugiere que términos como "objetivo" y "subjetivo" deben ser claramente definidos para evitar malentendidos, concluyendo que no existe una moralidad objetiva. Ambos textos tratan sobre la objetividad en valores y moralidad, pero el documento similar se centra en la precisión terminológica y la subjetividad de estos conceptos.

> Porcentaje de similitud: 65%

Respuesta al documento 5:

> El documento similar analiza la definición de "valores" en diferentes contextos, concluyendo que en el contexto científico, los valores se basan en hechos observables, mientras que en un sistema moral, los valores son principios considerados deseables. Argumenta que la ciencia no se basa en valores morales objetivos y cuestiona la posibilidad de una moralidad objetiva. Ambos textos abordan la relación entre valores, objetividad y moralidad, pero desde ángulos distintos, enfocándose en la precisión terminológica y la naturaleza subjetiva de la moralidad.

> Porcentaje de similitud: 70%

In [228]:
# Saving original document in .txt file.
with open(f"original.txt", "w") as text:
    print(len(newsgroups_train.data[documents[0].idx]))
    text.write(newsgroups_train.data[documents[0].idx])
        
# Saving similar documents in .txt files.
for i in range(len(documents)):
    with open(f"document{i}.txt", "w") as text:
        print(len(newsgroups_train.data[similarities[0][i]['idx']]))
        text.write(newsgroups_train.data[similarities[0][i]['idx']])


610
412
2715
369
1670
1968


In [223]:
print(f"Documento original (idx: {documents[0].idx} | tipo: [{documents[0].class_type}])\n")
print(newsgroups_train.data[documents[0].idx])
print(f"Documento similar 1 (idx: {similarities[0][0]['idx']} | similaridad: {100 * similarities[0][0]['sim']:.2f}% | tipo: [{similarities[0][0]['class_type']}])\n")
print(newsgroups_train.data[similarities[0][0]['idx']])
print(f"Documento similar 2 (idx: {similarities[0][1]['idx']} | similaridad: {100 * similarities[0][1]['sim']:.2f}% | tipo: [{similarities[0][1]['class_type']}])\n")
print(newsgroups_train.data[similarities[0][1]['idx']])
print(f"Documento similar 3 (idx: {similarities[0][2]['idx']} | similaridad: {100 * similarities[0][2]['sim']:.2f}% | tipo: [{similarities[0][2]['class_type']}])\n")
print(newsgroups_train.data[similarities[0][2]['idx']])
print(f"Documento similar 4 (idx: {similarities[0][3]['idx']} | similaridad: {100 * similarities[0][3]['sim']:.2f}% | tipo: [{similarities[0][3]['class_type']}])\n")
print(newsgroups_train.data[similarities[0][3]['idx']])
print(f"Documento similar 5 (idx: {similarities[0][4]['idx']} | similaridad: {100 * similarities[0][4]['sim']:.2f}% | tipo: [{similarities[0][4]['class_type']}])\n")
print(newsgroups_train.data[similarities[0][4]['idx']])


Documento original (idx: 8569 | tipo: [sci.space])

Forwarded from:
PUBLIC INFORMATION OFFICE
JET PROPULSION LABORATORY
CALIFORNIA INSTITUTE OF TECHNOLOGY
NATIONAL AERONAUTICS AND SPACE ADMINISTRATION
PASADENA, CALIF. 91109. (818) 354-5011

Contact:  Mary A. Hardin

FOR IMMEDIATE RELEASE                              April 15, 1993
#1506

     Scientists at NASA's Jet Propulsion Laboratory report the
successful flight of a balloon carrying instruments designed to
measure and study chemicals in the Earth's ozone layer.

     The April 3 flight from California's Barstow/Daggett Airport
reached an altitude of 37 kilometers (121,000 feet) and took
measurements as part of a program established to correlate data
with the Upper Atmosphere Research Satellite (UARS).  

     The data from the balloon flight will also be compared to
readings from the Atmospheric Trace Molecular Spectroscopy
(ATMOS) experiment which is currently flying onboard the shuttle
Discovery.

     "We launch these balloons

## Parte 2

Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

## Parte 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.